### Knock41: 基本的な Directory を生成しよう
> #### Master data と input data
> 明確な基準はないが、更新頻度によって仕訳するとよい。
> - 更新頻度-高: input data
> - 更新頻度-低: master data

In [1]:
# Directory path の定義
import os
data_dir = 'data'
input_dir = os.path.join(data_dir, '0_input')
output_dir = os.path.join(data_dir, '10_output')
master_dir = os.path.join(data_dir, '99_master_dir')
print(input_dir)

data\0_input


`os.mkdir()` と `os.makedirs()` の違い
- `os.makedirs()` は、再帰的に中間の Directory を自動で生成してくれる。

In [2]:
# Directory の生成
os.makedirs(input_dir, exist_ok=True)
os.makedirs(output_dir, exist_ok=True)
os.makedirs(master_dir, exist_ok=True)

### Knock42: 入力 Data の check機構を作ろう

In [3]:
# Master data の読み込み
import pandas as pd

m_area_file = 'm_area.csv'
m_store_file = 'm_store.csv'
m_area = pd.read_csv(os.path.join(master_dir, m_area_file))
m_store = pd.read_csv(os.path.join(master_dir, m_store_file))
m_area.head(3)

,area_cd,wide_area,narrow_area
0,TK,東京,東京
1,KN,神奈川,神奈川
2,CH,千葉,千葉


In [9]:
# 直接 File 名を指定せずに、変数を対象年月として File を定義
tg_ym = '202007'
target_file = 'tbl_order_' + tg_ym + '.csv'
target_data = pd.read_csv(os.path.join(input_dir, target_file))

- なるべく混乱を避けるために、File 名ではなく、年月を指定するようにする等の工夫を行なう。
- 毎月特定営業日に Data 更新を行なうなどが決まっている場合は、`datetime` 等で現在の年月を取得するのも良い。
- Data がない, File 名が違う場合, `FileNotFoundError` 等が発生し Program が止まるため間違いを確認できる。
- Error も起きずに自分が気づかないうちに意図しない処理が行なわれてしまい、間違った Report を配ってしまうことが一番、問題となる。
- 毎月変化していく Data は、必ず check 機構を取り入れる。
- Data の中身を出力することが Check 機構の基本だが Human error を起こす場合がある。
- 可能な限り、Error を意図的に発生させて、Program を Stop できる機構を入れることが望ましい。

In [5]:
# Data check 機構（正常動作時）
import datetime

max_date = pd.to_datetime(target_data['order_accept_date']).max()
min_data = pd.to_datetime(target_data['order_accept_date']).min()
max_str_date = max_date.strftime('%Y%m')
min_str_date = min_data.strftime('%Y%m')
if tg_ym == min_str_date and tg_ym == max_str_date:
    print("日付が一致しました")
else:
    raise Exception("日付が一致しません")

日付が一致しました


csv や Excel などのような Data は簡単に File 名を変更できてしまうため、Data の内容から check できる機構を入れておくことで間違いに気が付ける。

In [8]:
# Data check 機構（Error 動作時）
import datetime

max_date = pd.to_datetime(target_data['order_accept_date']).max()
min_data = pd.to_datetime(target_data['order_accept_date']).min()
max_str_date = max_date.strftime('%Y%m')
min_str_date = min_data.strftime('%Y%m')
if tg_ym == min_str_date and tg_ym == max_str_date:
    print("日付が一致しました")
else:
    raise Exception("日付が一致しません")

Exception: 日付が一致しません

Check 機構は、取り組んでいる Data によって違いが出てくる。
どういった規則で check を行なうべきか考えて構築する。

In [11]:
# DataFrame の各種初期化処理
def calc_delta(t):
    t1, t2 = t
    delta = t2 - t1
    return delta.total_seconds() / 60


def init_tran_df(trg_df):
    # 保守用店舗 Data の削除
    trg_df = trg_df.loc[trg_df['store_id'] != 999]

    trg_df = pd.merge(trg_df, m_store, on='store_id', how='left')
    trg_df = pd.merge(trg_df, m_area, on='area_cd', how='left')

    # Master にない Code に対応した文字列を設定
    trg_df.loc[trg_df['takeout_flag'] == 0, 'takeout_name'] = "デリバリー"

    trg_df.loc[trg_df['takeout_flag'] == 1, 'takeout_name'] = "お持ち帰り"

    trg_df.loc[trg_df['status'] == 0, 'status_name'] = "受付"
    trg_df.loc[trg_df['status'] == 1, 'status_name'] = "お支払済"
    trg_df.loc[trg_df['status'] == 2, 'status_name'] = "お渡し済"
    trg_df.loc[trg_df['status'] == 9, 'status_name'] = "キャンセル"

    trg_df.loc[:, 'order_date'] = pd.to_datetime(trg_df['order_accept_date']).dt.date

    # 配達までの時間を計算
    trg_df.loc[:, 'order_accept_datetime'] = pd.to_datetime(trg_df['order_accept_date'])
    trg_df.loc[:, 'delivered_datetime'] = pd.to_datetime(trg_df['delivered_date'])
    trg_df.loc[:, 'delta'] = trg_df[['order_accept_datetime', 'delivered_datetime']].apply(calc_delta, axis=1)

    return trg_df


# 当月分を初期化
target_data = init_tran_df(target_data)

In [3]:
# Master data の読み込み
import pandas as pd

m_area_file = 'm_area.csv'
m_store_file = 'm_store.csv'
m_area = pd.read_csv(os.path.join(master_dir, m_area_file))
m_store = pd.read_csv(os.path.join(master_dir, m_store_file))
m_area.head(3)

,area_cd,wide_area,narrow_area
0,TK,東京,東京
1,KN,神奈川,神奈川
2,CH,千葉,千葉


In [4]:
# 直接 File 名を指定せずに、変数を対象年月として File を定義
tg_ym = '202007'
target_file = 'tbl_order_' + tg_ym + '.csv'
target_data = pd.read_csv(os.path.join(input_dir, target_file))

- なるべく混乱を避けるために、File 名ではなく、年月を指定するようにする等の工夫を行なう。
- 毎月特定営業日に Data 更新を行なうなどが決まっている場合は、`datetime` 等で現在の年月を取得するのも良い。
- Data がない, File 名が違う場合, `FileNotFoundError` 等が発生し Program が止まるため間違いを確認できる。
- Error も起きずに自分が気づかないうちに意図しない処理が行なわれてしまい、間違った Report を配ってしまうことが一番、問題となる。
- 毎月変化していく Data は、必ず check 機構を取り入れる。
- Data の中身を出力することが Check 機構の基本だが Human error を起こす場合がある。
- 可能な限り、Error を意図的に発生させて、Program を Stop できる機構を入れることが望ましい。

In [5]:
# Data check 機構（正常動作時）
import datetime

max_date = pd.to_datetime(target_data['order_accept_date']).max()
min_data = pd.to_datetime(target_data['order_accept_date']).min()
max_str_date = max_date.strftime('%Y%m')
min_str_date = min_data.strftime('%Y%m')
if tg_ym == min_str_date and tg_ym == max_str_date:
    print("日付が一致しました")
else:
    raise Exception("日付が一致しません")

日付が一致しました


csv や Excel などのような Data は簡単に File 名を変更できてしまうため、Data の内容から check できる機構を入れておくことで間違いに気が付ける。